In [145]:
import numpy as np
import copy
import math  

In [173]:
def Sandpile_2D(size, crit_value, trails, boundary_con = False, adding_method = "position"):
    # basic sandpile model of given size (size by size matrix) with given critical value and trails
    # boundary_con is True walled model (sand is not removed) if False sand is removed from the sides
    # adding method; Where are the grains added using the add_sand function (position, random, normal)
    
    A = np.zeros((size,size))
    for k in range(trails):
        
        # add sand to the model
        add_sand(A, adding_method)
        
        # update the model simetaniously
        B = copy.deepcopy(A)
        for i in range(0, len(A)):
            for j in range(0, len(A[0])):
                if  A[i][j] - min(neigbours(B, [i,j])[0]) >= crit_value and B[i][j] >= 4:
                    
                    # implement the boundary condition and lower the pile at critical value
                    if boundary_con == True:
                        A[i][j] -= sum(neigbours(A, [i,j])[1])
                    else:
                        A[i][j] -= 4

                    # add to surrounding area
                    itter = 0
                    for adj in range(-1, 2, 2):
                        if neigbours(A, [i,j])[1][itter] == 1:
                            A[i + adj][j] += 1
                        itter += 1

                        if neigbours(A, [i,j])[1][itter] == 1:
                            A[i][j + adj] += 1
                        itter += 1
    return A

In [171]:
def neigbours(Matrix, cord):
    # find the neibour values of a given cell
    # Returns the values and gives the neigbours that exist given the dataset
    # in order: left, upper, right and down neigbouring cells.
    
    neigbours = list()
    real = np.zeros(4)
    
    if cord[0] >= 1:
        neigbours.append(Matrix[cord[0] - 1][cord[1]])
        real[0] += 1
    if cord[1] >= 1:
        neigbours.append(Matrix[cord[0]][cord[1] - 1])
        real[1] += 1
    
    if cord[0] < len(Matrix) - 1:
        neigbours.append(Matrix[cord[0] + 1][cord[1]])
        real[2] += 1
    if cord[1] < len(Matrix) - 1:
        neigbours.append(Matrix[cord[0]][cord[1] + 1])
        real[3] += 1
        
    return neigbours, real

In [170]:
def add_sand(Matrix, method, cords = [-1, -1], std = -1):
    # add value to matrix using various methods'
    # position: at given postion standard is center
    # random: at random position
    # normal: at a normal distribution with given stanardard deviation around given position
    # standard postion is center and standard deviation is 1/6 th of the matrix size (3 SD intervall covers 99.9% of the matrix)
    
    if method.lower() == "random":
        Matrix[np.random.randint(len(Matrix) - 1)][np.random.randint(len(Matrix[0]) - 1)] += 1
        return
    
    # set values of cordinates if no values are given
    if cords[0] == -1:
        cords[0] = math.floor(len(Matrix)/2)
    if cords[1] == -1:
        cords[1] = math.floor(len(Matrix)/2)
    
    if method.lower() == "position":
        print("this")

        Matrix[cords[0]][cords[1]] += 1
        return
    
    if method.lower() == "normal":
        # set value of standard deviation
        if std == -1:
            std = math.ceil(len(Matrix)/6)
            
        Matrix[np.random.normal(cords[0], std)][np.random.normal(cords[1], std)] += 1